In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import *
from nltk import word_tokenize
import itertools
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Выгрузка данных из датасета

In [5]:
categories = ['comp.windows.x', 'rec.sport.baseball', 'rec.sport.hockey']
remove = ['headers', 'footers', 'quotes']
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories, remove=remove)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42, categories=categories, remove=remove)

In [6]:
twenty_train = pd.DataFrame(twenty_train, columns=['data', 'target']).replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
twenty_test = pd.DataFrame(twenty_test, columns=['data', 'target']).replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)

In [7]:
def stemming(data):
    porter_stemmer = PorterStemmer()
    nltk_tokens = word_tokenize(data)
    line = ''
    for word in nltk_tokens:
        line += ' ' + porter_stemmer.stem(word)
    return line

twenty_train.insert(loc=1, column='data_stemmed', value=twenty_train['data'].apply(lambda text: stemming(text)))
twenty_test.insert(loc=1, column='data_stemmed', value=twenty_test['data'].apply(lambda text: stemming(text)))

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics

from sklearn.utils._testing import ignore_warnings 
from sklearn.exceptions import FitFailedWarning, ConvergenceWarning 

In [9]:
%%time
parameters = {
    'RAndomForestClassifier': {
        'vect__max_features': (1000,5000,10000),
        'vect__stop_words': ('english', None),
        'tfidf__use_idf': (True, False),
        'clf__n_neighbors': (1, 3, 5, 10),
        'clf__p': (1, 2)
    },
    'GaussianNB': {
        'vect__max_features': (1000,5000,10000),
        'vect__stop_words': ('english', None),
        'tfidf__use_idf': (True, False),
        'clf__criterion': ('gini', 'entropy'),
        'clf__max_depth': [*range(1,5,1), *range(5,101,20)]
    },
    'svm': [{
        'vect__max_features': (1000,5000,10000),
        'vect__stop_words': ('english', None),
        'tfidf__use_idf': (True, False),
        'clf__loss': ['squared_hinge'],
        'clf__penalty': ('l1', 'l2')
    },
        {
        'vect__max_features': (1000,5000,10000),
        'vect__stop_words': ('english', None),
        'tfidf__use_idf': (True, False),
        'clf__loss': ['hinge'],
        'clf__penalty': ['l2']
    }],
}

gs = {}
for clf, param in parameters.items():
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', eval(clf)())
    ])
    gs[clf] = GridSearchCV(text_clf, param, n_jobs=-1, error_score=0.0)
    gs[clf].fit(X = twenty_train['data'], y = twenty_train['target'])

/home/user/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to 0.0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/user/.local/lib/python3.8/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_classes.py", line 257, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_libl

CPU times: user 10.2 s, sys: 2.02 s, total: 12.2 s
Wall time: 6min 58s


In [10]:
for clf, param in parameters.items():
    predicted = gs[clf].predict(twenty_test['data'])
    print(metrics.classification_report(twenty_test.target, predicted, target_names=categories))

                    precision    recall  f1-score   support

    comp.windows.x       0.75      0.56      0.64       395
rec.sport.baseball       0.43      0.57      0.49       397
  rec.sport.hockey       0.51      0.48      0.49       399

          accuracy                           0.53      1191
         macro avg       0.56      0.53      0.54      1191
      weighted avg       0.56      0.53      0.54      1191

                    precision    recall  f1-score   support

    comp.windows.x       0.87      0.80      0.83       395
rec.sport.baseball       0.63      0.81      0.71       397
  rec.sport.hockey       0.82      0.64      0.72       399

          accuracy                           0.75      1191
         macro avg       0.77      0.75      0.75      1191
      weighted avg       0.77      0.75      0.75      1191

                    precision    recall  f1-score   support

    comp.windows.x       0.98      0.93      0.96       395
rec.sport.baseball       0.85    

In [11]:
r = {}
def highlight_max(x, color):

    return np.where(x == np.nanmax(x.to_numpy()), f"color: {color};", None)

total_style = pd.Series("font-weight: bold;", index=[1])

for clf, param in parameters.items():
    predicted = gs[clf].predict(twenty_test['data'])
    
    pd.DataFrame(gs[clf].cv_results_).to_excel('all' + clf + '.xlsx')
    pd.DataFrame(classification_report(predicted, twenty_test.target, output_dict=True)).to_excel('best' + clf + '.xlsx')
    